# 한국어 코퍼스 전처리 
- 데이터셋 : Korpora에서 로딩 
- 형태소 분석기 설정 
- 단어 사전 생성 

## [1] 모듈 로딩 및 데이터 준비 
---

In [58]:
# 모듈 로딩
from Korpora import Korpora # open Korean Dataset
from konlpy.tag import *    # 형태소 분석기
import spacy                # 형태소 분석기 

from torchtext.data.utils import get_tokenizer # pytorch 자연어 처리 모듈 
from torchtext.vocab import * # pytorch 자연어 처리 단어사전 관련 모듈 
from torch.utils.data import Dataset, DataLoader # pytorch Dataset 관련 모듈

import pandas as pd 
import numpy as np

In [59]:
# 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/anhyojun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/anhy

In [60]:
nsmc.train[0]

LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0)

In [61]:
nsmc.test[0]

LabeledSentence(text='굳 ㅋ', label=1)

In [62]:
# test 데이터셋을 DF로 로딩 
nsmc_df = pd.DataFrame(nsmc.test)
nsmc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


-> text가 feature, label이 target

## [2] 커스텀 데이터셋 생성
---

In [63]:
# 단어 사전을 생성 시 활용 
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.length = feature.shape[0] 

    def __len__(self):
        return self.length

    def __getitem__(self,index):
        return self.feature.iloc[index], self.label.iloc[index] # 행을 읽어야 하므로 iloc 사용 

In [64]:
# nsmc의 데이터셋 인스턴스 생성 
nsmc_ds = TextDataset(nsmc_df['text'], nsmc_df['label'])

In [65]:
DL = DataLoader(nsmc_ds, batch_size = 2)

# 첫 번째 배치 출력
for batch in DL:
    print(batch)
    break  # 첫 번째 배치만 출력

[('굳 ㅋ', 'GDNTOPCLASSINTHECLUB'), tensor([1, 0])]


In [35]:
for f, l in nsmc_ds:
    print(f,l)
    break # break 안 넣으면 노트북 죽음,,,,

굳 ㅋ 1


## [3] 텍스트 데이터 전처리 
---
- 토큰화 / 정제 (불용어, 구두점, 오타....)
- 단어 사전 생성 
- 문장 => 수치화 

In [36]:
# 토커나이즈 생성 

okt_tokenizer = get_tokenizer(Okt().morphs) # 함수 이름을 줘야지 함수 주소가 넘어감 
spacy_tokenizer = get_tokenizer('spacy', language='ko_core_news_sm')

In [37]:
# 텍스트 Dataset에서 처리할 데이터셋만 로딩해서 토큰화 진행하는 반복자 함수 
def generate_token(dataset):
    for text, label in dataset:
        yield okt_tokenizer(text)

In [38]:
# 단어 사전 생성 
unk_token = '<unk>'
pad_token = '<pad>'
nsmc_vocab = build_vocab_from_iterator(iterator=generate_token(nsmc_ds), specials=['<unk>']) # specials=['<unk>', '<pad>']
# 함수에 가서 한 개씩 처리하고 iterator에 줌 - 반복

nsmc_vocab.set_default_index(nsmc_vocab['<unk>']) # oov 토큰의 인덱스 

TypeError: build_vocab_from_iterator() got an unexpected keyword argument 'specials'

In [24]:
import torchtext

torchtext.__version__

'0.6.0'